In [1]:
#Read Data From CSV file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
df=pd.read_csv('NEWDATATABLE.csv', sep=',',header=None)
ratings = df.values

In [2]:
def train_test_generation(ratings):
    #Cretae train and test matrix
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in xrange(ratings.shape[0]):
        temp = list(ratings[user, :])
        temp_list = []
        count = 0
        # Choose top 10 known to be in test case
        for item in temp:
            if item != 99:
                temp_list.append(item)
        temp_list = sorted(temp_list, reverse=True)  
        while count < 10:
            count = count + 1
            Index_temp = temp.index(temp_list[count])
            #Set chosen be zero in training
            train [user, Index_temp] = 0.
            #Assign chosen rating to test case
            test[user, Index_temp] = ratings[user, Index_temp]
            ratings[user, Index_temp] = 0.
            temp = list(ratings[user, :])
            temp_list = []
            for item in temp:
                if item != 99:
                    temp_list.append(item)
            temp_list = sorted(temp_list, reverse=True)  
              
    return train, test

In [3]:
#Call the function and then assign train and test
train, test = train_test_generation(ratings)
trainMAE = train.copy()
#Masked for CF and not masked for SVD
train = np.ma.masked_where(train == 99, train)
test = np.ma.masked_where(test == 99, test)

In [4]:
def SGD_factorization(train, K, steps=100, alpha=0.0001, beta=0.02):
    N_user, N_item = train.shape
    #U and V are latent vectors
    U = np.random.rand(N_user,K)
    V = np.random.rand(N_item,K).T
    
    #Run 10 steps
    for step in xrange(steps):
        for i in xrange(N_user):
            for j in xrange(N_item):
                if train[i,j] > -10 and train[i,j] < 10:
                    eij = train[i][j] - np.dot(U[i,:],V[:,j])
                    U[i] = U[i] + alpha * (2 * eij * V[:,j] - beta * U[i])
                    V[:,j] = V[:,j] + alpha * (2 * eij * U[i] - beta * V[:,j])
    nR = np.dot(U,V)
    return nR

In [5]:
K = 10
prediction_SGD = SGD_factorization(train, K=10, steps=100, alpha=0.0001)

In [6]:
#Assign user_prediction or item_prediction to parameter
all_user_predicted_ratings = prediction_SGD

In [32]:
# 30 items for each user in test case and choose Top-N
#Pick unkonwn index
#evaluate recall and precise
unknown_index = []
exist_row = []
#discard
notexist_row = []
for indexI in range(24983):
    temp_row_index = np.where(trainMAE[indexI] == 99)
    if len(list(temp_row_index[0])) < 30 :
            notexist_row.append(indexI)
    else:
        temp_row_index_final = list(np.random.choice(temp_row_index[0], size = 30, replace = False))
        unknown_index.append(temp_row_index_final)
        exist_row.append(indexI)

In [33]:
#Find unknown rating for each user in training
from collections import defaultdict
combine = zip(exist_row, unknown_index)
row_number_result = defaultdict(list)
for indexI in range(len(combine)):
    row_temp = list()
    for item in combine[indexI][1]:
        row_temp.append(all_user_predicted_ratings[combine[indexI][0]][item])
    row_number_result[combine[indexI][0]].append(row_temp)

In [34]:
#Find known rating for each user in test
row_test_result = defaultdict(list)
for indexI in range(len(combine)):
    a = test[combine[indexI][0]]
    a = a[a != 0]
    a = list(a)
    row_test_result[combine[indexI][0]].append(a)

In [35]:
def recall_precise(index_matrix, test_index):
    recall_result = []
    precise_result = []
    N = 1
    recall_temp = 0
    precise_temp = 0
    
    for item in exist_row:
        temp_train_list = index_matrix[item][0]
        for item_test in row_test_result[item][0]:
            temp_train_list.append(item_test)
            temp_sort_list = sorted(temp_train_list, reverse = True)
            temp_index = temp_sort_list.index(item_test)
            if temp_index < N:
                count = 1
            else:
                count = 0 
            recall_temp = count
            precise_temp = float(count) / N
            recall_result.append(recall_temp)
            precise_result.append(precise_temp)
    return recall_result, precise_result

In [36]:
#Call two fcns to achieve to find
recall_each, precise_each = recall_precise(row_number_result, row_test_result)
recall_AVG = float(sum(recall_each)) / len(recall_each)
precise_AVG = float(sum(precise_each)) / len(precise_each)
print 'Recall: ' + str(recall_AVG)
print 'Precise: ' + str(precise_AVG)

Recall: 0.998859458854
Precise: 0.0399543783541
